In [1]:
import pandas as pd
import requests

In [2]:
def get_explore_url(lat, lng, radius):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    return url

In [3]:
CLIENT_ID = 'CCZZRQFMN0ASTRIFCSGDCSDH4NLUA2C1PDV1DNZLX0KK4AXY'
CLIENT_SECRET = 'UTWNBZO55ZL4NFTQZ5UAKHPEZQ0M1PRZHXCHKHLYNPR2UZHT'
VERSION = '20180605'
LIMIT = 100

In [4]:
# ORIGIN code for capital data download:

# capitals['explore_url'] = capitals.apply(lambda row: get_explore_url(row['Capital Latitude'],
#                                                                  row['Capital Longitude'], 5000), axis=1)
# capitals['json'] = capitals['explore_url'].apply(lambda x: requests.get(x).json())
# capitals["response"] = capitals['json'].apply(lambda x: x['response'])


# due to limited number of queries and downloadtime I use a file contains once downloaded data.

capitals = pd.read_pickle('shot_two.pkl')
capitals.head()

,Country Name,Capital Name,Capital Latitude,Capital Longitude,Country Code,Continent Name,explore_url,json,response
1,Afghanistan,Kabul,34.51666667,69.183333,AF,Asia,https://api.foursquare.com/v2/venues/explore?&...,"{'meta': {'code': 200, 'requestId': '5f1405736...",{'suggestedFilters': {'header': 'Tap to show:'...
2,Aland Islands,Mariehamn,60.116667,19.9,AX,Europe,https://api.foursquare.com/v2/venues/explore?&...,"{'meta': {'code': 200, 'requestId': '5f1408ed6...",{'suggestedFilters': {'header': 'Tap to show:'...
3,Albania,Tirana,41.31666667,19.816667,AL,Europe,https://api.foursquare.com/v2/venues/explore?&...,"{'meta': {'code': 200, 'requestId': '5f14062f6...",{'suggestedFilters': {'header': 'Tap to show:'...
4,Algeria,Algiers,36.75,3.05,DZ,Africa,https://api.foursquare.com/v2/venues/explore?&...,"{'meta': {'code': 200, 'requestId': '5f1407e04...",{'suggestedFilters': {'header': 'Tap to show:'...
5,American Samoa,Pago Pago,-14.26666667,-170.7,AS,Australia,https://api.foursquare.com/v2/venues/explore?&...,"{'meta': {'code': 200, 'requestId': '5f14080e5...","{'headerLocation': 'Pago Pago', 'headerFullLoc..."


In [5]:
life_expectancy = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy')[0]
life_expectancy = life_expectancy[[('Countries and regions', 'Countries and regions'),
                                   ('Life expectancy at birth (in years)', 'Overall')]]
life_expectancy = life_expectancy.T.reset_index(drop=True).T.rename(columns={0: 'Countries and regions',
                                                                             1: 'Life expectancy'})
life_expectancy.head()

,Countries and regions,Life expectancy
0,Hong Kong,84.7
1,Japan,84.5
2,Singapore,83.8
3,Italy,83.6
4,Spain,83.4


In [6]:
gdp_per_capita = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)_per_capita')[3]
gdp_per_capita.head()

,Rank,Country/Territory,Int$
0,—,Macau,129103
1,1,Luxembourg (EU),121293
2,2,Singapore,101376
3,3,Qatar,96491
4,4,Ireland (EU),88241


In [7]:
capitals_population = pd.read_html('https://en.wikipedia.org/wiki/List_of_national_capitals_by_population')[1]
capitals_population.rename(columns={'Capital': 'Capital Name'}, inplace=True)
capitals_population.to_csv('capitals_population.csv')
capitals_population.head()

,Rank,Country/Territory,Capital Name,Population,Year,% ofcountry'spopulation
0,1,China,Beijing,"21,542,000[1]",2010,1.5%
1,2,India,New Delhi,14200004,2011,1.05%
2,3,Japan,Tokyo,"13,929,286[2]",2017,11.03%
3,4,Russia,Moscow,"12,506,468[3]",2011,8.52%
4,5,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%


In [8]:
capitals_population['Population'] = capitals_population['Population'].apply(lambda x: 
                                                                            x.split('[')[0].replace(',',''))
capitals_population['Population'] = capitals_population['Population'].apply(lambda x: 
                                                                            x.split(' (')[0])
capitals_population = capitals_population[capitals_population['Population'] != 'n/a']
capitals_population['Population'] = capitals_population['Population'].apply(float)

# Country name unification

In [9]:
life_expectancy['Countries and regions'].replace({"People's Republic of China": 'China',
                                                  'Brunei': 'Brunei Darussalam', 'Eswatini': 'Swaziland',
                                                  'Congo, Democratic Republic of the': 'Democratic Republic of the Congo',
                                                  "Côte d'Ivoire": 'Cote d’Ivoire', 'Gambia': 'The Gambia',
                                                  'Congo, Republic of': 'Republic of Congo', 
                                                  'Timor Leste':'Timor-Leste', 'Cabo Verde': 'Cape Verde',
                                                  'North Macedonia': 'Macedonia'
                                                  }, inplace=True)

life_expectancy.rename(columns={'Countries and regions': 'Country Name'}, inplace=True)

In [10]:
gdp_per_capita['Country/Territory'] = gdp_per_capita['Country/Territory'].apply(lambda x: x.split(' (')[0])
gdp_per_capita['Country/Territory'].replace({'Brunei': 'Brunei Darussalam', 
                                             'Congo, Democratic Republic of the': 'Democratic Republic of the Congo',
                                             "Côte d'Ivoire": 'Cote d’Ivoire', 'Gambia, The': 'The Gambia',
                                             'Congo, Republic of the': 'Republic of Congo', 
                                             'Bahamas, The': 'Bahamas', 'Curaçao': 'CuraÃ§ao',
                                             'Korea, South': 'South Korea', 'Eswatini': 'Swaziland',
                                             'São Tomé and Príncipe': 'Sao Tome and Principe',
                                             'Micronesia, Federated States of': 'Federated States of Micronesia',
                                             'North Macedonia': 'Macedonia'}, inplace=True)
gdp_per_capita.rename(columns={'Country/Territory': 'Country Name'}, inplace=True)

In [11]:
capitals_population['Country/Territory'] = capitals_population['Country/Territory'].apply(lambda x: 
                                                                                          x.split(' (')[0])
capitals_population['Country/Territory'].replace({'Brunei': 'Brunei Darussalam', 
                     'DR Congo': 'Democratic Republic of the Congo',
                     'Ivory Coast': 'Cote d’Ivoire', 'Åland': 'Aland Islands',
                     'Congo': 'Republic of Congo', 'Cocos': 'Cocos Islands',
                     'Curaçao': 'CuraÃ§ao', 'East Timor': 'Timor-Leste',
                     'Eswatini': 'Swaziland', 'Saint Barthélemy': 'Saint Barthelemy',
                     'United States Virgin Islands': 'US Virgin Islands',
                     'Collectivity of Saint Martin': 'Saint Martin',
                     'South Georgia and the South Sandwich Islands': 'South Georgia and South Sandwich Islands',
                     'São Tomé and Príncipe': 'Sao Tome and Principe', 
                     'Sahrawi Arab Democratic Republic': 'Western Sahara',
                     'North Macedonia': 'Macedonia'}, inplace=True) 
capitals_population.rename(columns={'Country/Territory': 'Country Name'}, inplace=True)

In [12]:
capitals_population.rename(columns={'Capital Name': 'Capital Name pop'}, inplace=True)

In [13]:
capitals.set_index('Country Name', inplace=True)
life_expectancy.set_index('Country Name', inplace=True)
gdp_per_capita.set_index('Country Name', inplace=True)
capitals_population.set_index('Country Name', inplace=True)

In [14]:
to_analyse = pd.concat([capitals, capitals_population, life_expectancy, gdp_per_capita], axis=1, sort=False)
to_analyse.drop(columns=['Year', 'explore_url', 'json', 'Rank', 
                         'Country Code', "% ofcountry'spopulation"], inplace=True)

# Capital names comparison {capitals vs capitals_population}

In [15]:
to_analyse['Capital Name pop'].replace({"St. John's": "Saint John’s", 'Bogotá': 'Bogota', 'San José': 'San Jose',
                                        'Brasília': 'Brasilia', 'Yaoundé': 'Yaounde', "N'Djamena": "N’Djamena",
                                        'Djibouti (city)': 'Djibouti', 'Tórshavn': 'Torshavn', 
                                        'Hagåtña': 'Hagatna', 'St. Peter Port': 'Saint Peter Port',
                                        'Reykjavík': 'Reykjavik', 'Jerusalem[43]': 'Jerusalem',
                                        'Nur-Sultan': 'Astana', 'Prishtina': 'Pristina', 'Malé': 'Male',
                                        'Nuuk (Godthåb)': 'Nuuk', "St. George's": 'Saint George’s', 
                                        'Chişinău': 'Chisinau', 'Yaren (de facto)': 'Yaren', 'Nouméa': 'Noumea',
                                        'Quito (de jure) Guayaquil (de facto) seat-of-government': 'Quito',
                                        'Kingston (de facto)': 'Kingston', 'Asunción': 'Asuncion', 
                                        'City of San Marino': 'San Marino', 'Berne (de facto)': 'Bern', 
                                        'Lomé': 'Lome', "Nukuʻalofa": "Nuku’alofa", 'Kiev': 'Kyiv',
                                        'Port Vila': 'Port-Vila', 'Vatican City (city-state)': 'Vatican City',
                                        'El Aaiún (claimed)Tifariti (factual)': 'El-AaiÃºn', "Sana'a": 'Sanaa',
                                        'São Tomé': 'Sao Tome'
                                       }, inplace=True) 

In [16]:
# Number of cities are abandoned due to capital ambiguity
to_analyse[(to_analyse['Capital Name'] != to_analyse['Capital Name pop']) & 
           ~to_analyse['Capital Name'].isna()] [['Capital Name', 'Capital Name pop']]

,Capital Name,Capital Name pop
Bolivia,La Paz,Sucre
British Indian Ocean Territory,Diego Garcia,NaN
Burundi,Bujumbura,Gitega
Christmas Island,The Settlement,Flying Fish Cove
French Southern and Antarctic Lands,Port-aux-FranÃ§ais,NaN
Montserrat,Plymouth,Brades (de facto)
Myanmar,Rangoon,Naypyidaw[45]
Norfolk Island,Kingston,NaN
Northern Cyprus,North Nicosia,NaN
Palau,Melekeok,Ngerulmud


In [17]:
to_analyse = to_analyse[to_analyse['Capital Name'] == to_analyse['Capital Name pop']]
to_analyse.drop(columns=['Capital Name pop'], inplace=True)
to_analyse.dropna(inplace=True)

In [18]:
to_analyse.head()

,Capital Name,Capital Latitude,Capital Longitude,Continent Name,response,Population,Life expectancy,Int$
Afghanistan,Kabul,34.51666667,69.183333,Asia,{'suggestedFilters': {'header': 'Tap to show:'...,3140853.0,64.5,2293.0
Algeria,Algiers,36.75,3.05,Africa,{'suggestedFilters': {'header': 'Tap to show:'...,3415811.0,76.7,11820.0
Angola,Luanda,-8.833333333,13.216667,Africa,{'suggestedFilters': {'header': 'Tap to show:'...,2453779.0,60.8,6929.0
Antigua and Barbuda,Saint John’s,17.11666667,-61.85,North America,{'suggestedFilters': {'header': 'Tap to show:'...,22679.0,76.9,22817.0
Argentina,Buenos Aires,-34.58333333,-58.666667,South America,{'suggestedFilters': {'header': 'Tap to show:'...,2891082.0,76.5,22947.0


In [19]:
def getNearbyVenues(df):
    
    venues_list=[]
    for row_num, row in df.iterrows():
            
        results = row['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            row['Capital Name'], 
            row['Capital Latitude'], 
            row['Capital Longitude'], 
            row['Population'],
            row['Int$'],
            row['Life expectancy'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Capital Name', 
                  'Capital Latitude', 
                  'Capital Longitude', 
                  'Population',
                  "GDP per capita [int$]",
                  'Life expectancy',           
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
capital_venues = getNearbyVenues(to_analyse)
capital_venues.drop(columns=['Venue Latitude', 'Venue Longitude'], inplace=True)
capital_venues.head()

,Capital Name,Capital Latitude,Capital Longitude,Population,GDP per capita [int$],Life expectancy,Venue Name,Venue Category
0,Kabul,34.51666667,69.183333,3140853.0,2293.0,64.5,Kabul Serena Hotel,Hotel
1,Kabul,34.51666667,69.183333,3140853.0,2293.0,64.5,Ciano ISAF,Pizza Place
2,Kabul,34.51666667,69.183333,3140853.0,2293.0,64.5,Park Star Hotel,Hotel
3,Kabul,34.51666667,69.183333,3140853.0,2293.0,64.5,Barg Continental,Afghan Restaurant
4,Kabul,34.51666667,69.183333,3140853.0,2293.0,64.5,İstanbul Restaurant,Turkish Restaurant
